In [1]:
!pip install albumentations
import albumentations as A

/opt/conda/lib/python3.10/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.0 (you have 1.4.21). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
import os
import shutil
import time
import pprint

import torch
import torch.nn as nn
import torch.autograd.variable as Variable

from math import sqrt
from numpy.random import seed
from numpy.random import randn
from numpy import mean
from scipy.stats import sem
from scipy.stats import t
import numpy as np
from collections import OrderedDict

class GaussianNoise(nn.Module):
    def __init__(self, batch_size, input_shape=(3, 84, 84), std=0.05):
        super(GaussianNoise, self).__init__()
        self.shape = (batch_size,) + input_shape
        self.noise = Variable(torch.zeros(self.shape).cuda())
        self.std = std

    def forward(self, x, std=0.15):
        noise = Variable(torch.zeros(x.shape).cuda())
        noise = noise.data.normal_(0, std=std)
        return x + noise


def set_gpu(x):
    os.environ['CUDA_VISIBLE_DEVICES'] = x
    print('using gpu:', x)


def clone(tensor):
    """Detach and clone a tensor including the ``requires_grad`` attribute.

    Arguments:
        tensor (torch.Tensor): tensor to clone.
    """
    cloned = tensor.clone()#tensor.detach().clone()
    # cloned.requires_grad = tensor.requires_grad
    # if tensor.grad is not None:
    #     cloned.grad = clone(tensor.grad)
    return cloned

def clone_state_dict(state_dict):
    """Clone a state_dict. If state_dict is from a ``torch.nn.Module``, use ``keep_vars=True``.

    Arguments:
        state_dict (OrderedDict): the state_dict to clone. Assumes state_dict is not detached from model state.
    """
    return OrderedDict([(name, clone(param)) for name, param in state_dict.items()])

def ensure_path(path):
    if os.path.exists(path):
        if input('{} exists, remove? ([y]/n)'.format(path)) != 'n':
            shutil.rmtree(path)
            os.mkdir(path)
    else:
        os.mkdir(path)

class Averager():
    def __init__(self):
        self.n = 0
        self.v = 0

    def add(self, x):
        self.v = (self.v * self.n + x) / (self.n + 1)
        self.n += 1

    def item(self):
        return self.v

def count_acc(logits, label):
    pred = torch.argmax(logits, dim=1)
    return (pred == label).type(torch.cuda.FloatTensor).mean().item()

def dot_metric(a, b):
    return torch.mm(a, b.t())

def euclidean_metric(a, b):
    n = a.shape[0]
    m = b.shape[0]
    a = a.unsqueeze(1).expand(n, m, -1)
    b = b.unsqueeze(0).expand(n, m, -1)
    logits = -((a - b)**2).sum(dim=2)
    return logits

class Timer():

    def __init__(self):
        self.o = time.time()

    def measure(self, p=1):
        x = (time.time() - self.o) / p
        x = int(x)
        if x >= 3600:
            return '{:.1f}h'.format(x / 3600)
        if x >= 60:
            return '{}m'.format(round(x / 60))
        return '{}s'.format(x)

_utils_pp = pprint.PrettyPrinter()
def pprint(x):
    _utils_pp.pprint(x)

def l2_loss(pred, label):
    return ((pred - label)**2).sum() / len(pred) / 2

def set_protocol(data_path, protocol, test_protocol, subset=None):
    train = []
    val = []
    all_set = ['shn', 'hon', 'clv', 'clk', 'gls', 'scl', 'sci', 'nat', 'shx', 'rel']
    if subset is not None:
        train.append(data_path + '/crops_' + subset + '/')
        val.append(data_path + '/crops_' + subset + '/')
    if protocol == 'p1':
        for i in range(3):
            train.append(data_path + '/crops_' + all_set[i])
    elif protocol == 'p2':
        for i in range(3, 6):
            train.append(data_path + '/crops_' + all_set[i])
    elif protocol == 'p3':
        for i in range(6, 8):
            train.append(data_path + '/crops_' + all_set[i])
    elif protocol == 'p4':
        for i in range(8, 10):
            train.append(data_path + '/crops_' + all_set[i])

    if test_protocol == 'p1':
        for i in range(3):
            val.append(data_path + '/crops_' + all_set[i])
    elif test_protocol == 'p2':
        for i in range(3, 6):
            val.append(data_path + '/crops_' + all_set[i])
    elif test_protocol == 'p3':
        for i in range(6, 8):
            val.append(data_path + '/crops_' + all_set[i])
    elif test_protocol == 'p4':
        for i in range(8, 10):
            val.append(data_path + '/crops_' + all_set[i])
    return train, val

def flip(x, dim):
    dim = x.dim() + dim if dim < 0 else dim
    return x[tuple(slice(None, None) if i != dim
             else torch.arange(x.size(i)-1, -1, -1).long()
             for i in range(x.dim()))]

def perturb(data):
    randno = np.random.randint(0, 5)
    if randno == 1:
        return torch.cat((data, data.flip(3)), dim=0)
    elif randno == 2: #180
        return torch.cat((data, data.flip(2)), dim=0)
    elif randno == 3: #90
        return torch.cat((data, data.transpose(2,3)), dim=0)
    else:
        return torch.cat((data, data.transpose(2, 3).flip(3)), dim=0)

# Data Manager

In [3]:
cd "/kaggle/input/ui-smells-dataset-separate/ui-smell-dataset-filtered/ui-smell-dataset-filtered"

/kaggle/input/ui-smells-dataset-separate/ui-smell-dataset-filtered/ui-smell-dataset-filtered


In [4]:
import os.path as osp
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms

class UiSmell(Dataset):

    def __init__(self, setname, img_path, category):
        # Dynamically set the path to the correct category's materials folder
        root_path = osp.join('./', category, 'materials')  # e.g., './bare/materials', './list/materials'
        csv_path = osp.join(root_path, setname + '.csv')
        
        # Read the CSV file
        lines = [x.strip() for x in open(csv_path, 'r').readlines()][1:]  # Skip the header
        

        IMG_PATH = osp.join('./', category)  # Path to images in the specific category folder
        data = []
        label = []

        # Dictionary to map string labels to numeric labels
        label_map = {}
        label_counter = 0

        # Parse CSV file
        for line in lines:
            name, lbl = line.split(',')  # Split filename and label

            # Map string label to numeric label
            if lbl not in label_map:
                label_map[lbl] = label_counter
                label_counter += 1

            path = osp.join(IMG_PATH, name)  # Full path to the image
            data.append(path)
            label.append(label_map[lbl])  # Convert label to numeric

        self.data = data
        self.label = label
        self.label_map = label_map  # Store the label map for reference


        # Define image transformations
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, i):
        path, label = self.data[i], self.label[i]

        # Open and process the image
        img = Image.open(path).convert('RGB')
        
        img = img.resize((84, 84)).convert('RGB')  # Resize to (84, 84)
        
        image = self.transform(img)

        return image, label


# Sampler

In [5]:
import torch
import numpy as np


class CategoriesSampler():

    def __init__(self, label, n_batch, n_cls, n_per):
        self.n_batch = n_batch
        self.n_cls = n_cls
        self.n_per = n_per

        label = np.array(label)
        self.m_ind = []
        for i in range(max(label) + 1):
            ind = np.argwhere(label == i).reshape(-1)
            ind = torch.from_numpy(ind)
            if len(ind) > 4:
                self.m_ind.append(ind)

    def __len__(self):
        return self.n_batch
    
    def __iter__(self):
        for i_batch in range(self.n_batch):
            batch = []
            classes = torch.randperm(len(self.m_ind))[:self.n_cls]
            for c in classes:
                l = self.m_ind[c]
                pos = torch.randperm(len(l))[:self.n_per]
                batch.append(l[pos])
            batch = torch.stack(batch).t().reshape(-1)
            #for i in range(1000):
            yield batch

# Convnet

In [6]:
!pip install torchsummary

# !pip install efficientnet_pytorch

In [7]:
pip install efficientnet_pytorch==0.7.0


  Preparing metadata (setup.py) ... - done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.0-py3-none-any.whl size=16016 sha256=560c25bc400083e29ebbdede7086bd8e49914fa585eb6e0e0e13f50a0567207e
  Stored in directory: /root/.cache/pip/wheels/60/d1/6f/0122987202bfc9c2095be04455ad4f26bc6c56d57ae69ba454
Successfully built efficientnet_pytorch
Note: you may need to restart the kernel to use updated packages.


In [8]:
import efficientnet_pytorch
print(efficientnet_pytorch.__version__)


0.7.0


In [9]:
from torchsummary import summary
import torchvision.models as models
import torch.nn as nn
import torch
import numpy as np

def BasicBlock():
    return 0

def conv_block(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, padding=1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(),
        # nn.Hardswish(),
        nn.MaxPool2d(2)
    )

class ConvNet(nn.Module):

    def __init__(self, x_dim=3, hid_dim=64, z_dim=64):
        super().__init__()
        self.encoder = nn.Sequential(
            conv_block(x_dim, hid_dim),
            conv_block(hid_dim, hid_dim),
            conv_block(hid_dim, hid_dim),
            conv_block(hid_dim, z_dim),
        )
        self.out_channels = 1600

    def forward(self, x):
        x = self.encoder(x)
        return x.view(x.size(0), -1)


from efficientnet_pytorch import EfficientNet
class Efnet_do(nn.Module):

    def __init__(self, ver='b0', x_dim=3, hid_dim=64, z_dim=64):
        super().__init__()
        self.encoder = EfficientNet.from_pretrained('efficientnet-'+ver, include_top = False)
        self.drop = nn.Dropout(0.5)
        self.out_channels = 1600

    def forward(self, x):
        x = self.encoder(x)
        return self.drop(x.view(x.size(0), -1))

class Efnet_nopre(nn.Module):

    def __init__(self, ver='b0', x_dim=3, hid_dim=64, z_dim=64):
        super().__init__()
        self.encoder = EfficientNet.from_name('efficientnet-'+ver, include_top = False)
        self.out_channels = 1600

    def forward(self, x):
        x = self.encoder(x)
        return x.view(x.size(0), -1)
        
class Efnet(nn.Module):

    def __init__(self, ver='b0', x_dim=3, hid_dim=64, z_dim=64):
        super().__init__()
        self.encoder = EfficientNet.from_pretrained('efficientnet-'+ver, include_top = False)
        self.out_channels = 1600

    def forward(self, x):
        x = self.encoder(x)
        return x.view(x.size(0), -1)

        
import torchvision.models as models
class Mbnet_nopre(nn.Module):

    def __init__(self, x_dim=3, hid_dim=64, z_dim=64):
        super().__init__()
        self.encoder = models.mobilenet_v3_large(pretrained=False)
        self.out_channels = 1600

    def forward(self, x):
        x = self.encoder(x)
        return x.view(x.size(0), -1)

class Mbnet(nn.Module):

    def __init__(self, x_dim=3, hid_dim=64, z_dim=64):
        super().__init__()
        self.encoder = models.mobilenet_v3_large(pretrained=True)
        self.out_channels = 1600

    def forward(self, x):
        x = self.encoder(x)
        return x.view(x.size(0), -1)


# Classifier

In [10]:
import torch
import torch.nn as nn

class Subspace_Projection(nn.Module):
    def __init__(self, num_dim=2):
        super().__init__()
        self.num_dim = num_dim

    def create_subspace(self, supportset_features, class_size, sample_size):
        all_hyper_planes = []
        means = []
        for ii in range(class_size):
            num_sample = sample_size
            all_support_within_class_t = supportset_features[ii]
            meann = torch.mean(all_support_within_class_t, dim=0)
            means.append(meann)
            all_support_within_class_t = all_support_within_class_t - meann.unsqueeze(0).repeat(num_sample, 1)
            all_support_within_class = torch.transpose(all_support_within_class_t, 0, 1)
            # print(all_support_within_class.double())
            uu, s, v = torch.svd(all_support_within_class.double(), some=False)
            uu = uu.float()
            all_hyper_planes.append(uu[:, :self.num_dim])

        all_hyper_planes = torch.stack(all_hyper_planes, dim=0)
        means = torch.stack(means)

        if len(all_hyper_planes.size()) < 3:
            all_hyper_planes = all_hyper_planes.unsqueeze(-1)

        return all_hyper_planes, means


    def projection_metric(self, target_features, hyperplanes, mu):
        eps = 1e-12
        batch_size = target_features.shape[0]
        class_size = hyperplanes.shape[0]

        similarities = []

        discriminative_loss = 0.0

        for j in range(class_size):
            h_plane_j =  hyperplanes[j].unsqueeze(0).repeat(batch_size, 1, 1)
            target_features_expanded = (target_features - mu[j].expand_as(target_features)).unsqueeze(-1)
            projected_query_j = torch.bmm(h_plane_j, torch.bmm(torch.transpose(h_plane_j, 1, 2), target_features_expanded))
            projected_query_j = torch.squeeze(projected_query_j) + mu[j].unsqueeze(0).repeat(batch_size, 1)
            projected_query_dist_inter = target_features - projected_query_j

            #Training per epoch is slower but less epochs in total
            query_loss = -torch.sqrt(torch.sum(projected_query_dist_inter * projected_query_dist_inter, dim=-1) + eps) # norm ||.||

            #Training per epoch is faster but more epochs in total
            #query_loss = -torch.sum(projected_query_dist_inter * projected_query_dist_inter, dim=-1) # Squared norm ||.||^2

            similarities.append(query_loss)

            for k in range(class_size):
                if j != k:
                   temp_loss = torch.mm(torch.transpose(hyperplanes[j], 0, 1), hyperplanes[k]) ## discriminative subspaces (Conv4 only, ResNet12 is computationally expensive)
                   discriminative_loss = discriminative_loss + torch.sum(temp_loss*temp_loss)

        similarities = torch.stack(similarities, dim=1)

        return similarities, discriminative_loss


# Training

In [11]:
import os
import torch
import argparse
import numpy as np
import os.path as osp
from datetime import datetime
import torch.nn.functional as F
from torch.utils.data import DataLoader

# SET MODEL NAME
# modelname = 'cnn'
# modelname = 'mbnet'
# modelname = 'mbnet_no'
# modelname = 'efnetb1'
# modelname = 'efnetb1_no'
# modelname = 'efnetb0'
# modelname = 'efnetb2'
modelname = 'efnetb3'
# modelname = 'efnetb4'
# modelname = 'efnetb5'
# modelname = 'efnetb1_do'
# modelname = 'efnetb1aug'
# modelname = 'efnetb1_doaug'

args = {}
args['num_sampler'] = 100
args['max-epoch'] = 20
args['save-epoch'] = 5
args['shot'] = 5
args['query'] = 5
args['train-way'] = 5
args['test-way'] = 5

args['data-path'] = '/kaggle/input/ui-smells-dataset-separate/ui-smell-dataset-filtered/ui-smell-dataset-filtered'
args['gpu'] = '0'
args['lamb'] = 0.03
args['lr'] = 0.001
args['subspace-dim'] = args['shot']-1
set_gpu(args['gpu'])

txt = str(datetime.now())
txt = '_'.join([modelname+f"-{args['shot']}-{args['lamb']}-{args['lr']}-{args['num_sampler']}", txt[:4], txt[5:7], txt[8:10], txt[11:13], txt[14:16]])
args['save-path'] = '/kaggle/working/save/'+txt

# MODEL BUILDER
model = {
    # 'cnn': ConvNet(),
    # 'mbnet_no': Mbnet_nopre(),
    # 'mbnet': Mbnet(),
    # 'efnetb1_no': Efnet_nopre(ver='b1'),
    # 'efnetb0': Efnet(ver='b0'),
    # 'efnetb1': Efnet(ver='b1'),
    # 'efnetb2': Efnet(ver='b2'),
    'efnetb3': Efnet(ver='b3'),
    # 'efnetb4': Efnet(ver='b4'),
    # 'efnetb5': Efnet(ver='b5'),
    # 'efnetb1_do': Efnet_do(ver='b1'),
}[modelname.replace('aug','')].cuda()


using gpu: 0


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b3-5fb5a3c3.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b3-5fb5a3c3.pth
100%|██████████| 47.1M/47.1M [00:00<00:00, 188MB/s]


Loaded pretrained weights for efficientnet-b3


In [12]:
import torch
from torch.utils.data import DataLoader
import numpy as np

# # Define the categories
# categories = ['bare', 'form', 'gallery', 'list', 'login',
#                 'menu', 'modal', 'news', 'other', 'profile', 
#                 'settings', 'terms', 'tutorial']

# # Specify categories explicitly for train and validation
# train_categories = [category for category in categories if category not in ['login', 'gallery', 'list']]
# val_categories = ['login', 'gallery']

# Define the categories
categories = ['bare', 'form', 'menu', 'modal', 'news', 'other', 'profile', 
                'settings', 'terms', 'tutorial', 'login', 'gallery', 'list']

# Specify categories explicitly for train and validation
val_categories = ['form', 'tutorial']
test_category = 'login'
train_categories = [category for category in categories if category not in val_categories+[test_category]]

# Initialize DataLoader lists
train_loaders = []
val_loaders = []

# Training DataLoader creation for each category
for category in train_categories:
    trainset = UiSmell('train', args['data-path'], category=category)
    train_sampler = CategoriesSampler(trainset.label, args['num_sampler'], args['train-way'], args['shot'] + args['query'])
    train_loader = DataLoader(dataset=trainset, batch_sampler=train_sampler, shuffle=False)
    train_loaders.append(train_loader)

# Validation DataLoader creation for each category
for category in val_categories:
    valset = UiSmell('val', args['data-path'], category=category)
    val_sampler = CategoriesSampler(valset.label, args['num_sampler'], args['test-way'], args['shot'] + args['query'])
    val_loader = DataLoader(dataset=valset, batch_sampler=val_sampler, shuffle=False)
    val_loaders.append(val_loader)

# Create dataset and loader for testing (using only 'list' category)
testset = UiSmell('test', args['data-path'], category=test_category)
test_sampler = CategoriesSampler(testset.label, args['num_sampler'], args['test-way'], args['shot'] + args['query'])
test_loader = DataLoader(dataset=testset, batch_sampler=test_sampler, shuffle=False)


In [13]:
from tqdm import tqdm
projection_pro = Subspace_Projection(num_dim=args['subspace-dim'])

optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])

if args['shot'] > 1:
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
else:
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

if args['shot'] == 1:
    shot_num = 2
    args['subspace-dim'] = 1
else:
    shot_num = args['shot']

In [14]:
# TRAINING PHASE
if not os.path.exists(args['save-path']):
    os.makedirs(args['save-path'])
torch.save(val_sampler, args['save-path']+'/support-set.pt')

def save_model(name):
    if not os.path.exists(args['save-path']):
        os.makedirs(args['save-path'])
    torch.save(model.state_dict(), osp.join(args['save-path'], name + '.pth'))

# Initialize trlog with category-based tracking
trlog = {}
for category in categories:
    trlog[category] = {
        'train_loss': [],
        'val_loss': [],
        'test_loss': [],
        'train_acc': [],
        'val_acc': [],
        'test_acc': [],
    }
trlog['max_acc'] = 0.0
trlog['max_epoch'] = 0

timer = Timer()

for epoch in range(1, args['max-epoch'] + 1):
    lr_scheduler.step()
    
    model.train()
    
    # Initialize Averager for training loss and accuracy
    tl = Averager()
    ta = Averager()

    # Loop over each train_loader and category
    for category, train_loader in zip(train_categories, train_loaders):
        for i, batch in tqdm(enumerate(train_loader, 1), desc=f"Training {category}"):
            data, _ = [_.cuda() for _ in batch]
            p = args['shot'] * args['train-way']
            qq = p + args['query'] * args['train-way']
            data_shot, data_query = data[:p], data[p:qq]

            if args['shot'] == 1:
                data_shot = torch.cat((data_shot, flip(data_shot, 3)), dim=0)

            proto = model(data_shot)
            proto = proto.reshape(shot_num, args['train-way'], -1)
            proto = torch.transpose(proto, 0, 1)
            hyperplanes, mu = projection_pro.create_subspace(proto, args['train-way'], shot_num)

            label = torch.arange(args['train-way']).repeat(args['query'])
            label = label.type(torch.cuda.LongTensor)

            logits, discriminative_loss = projection_pro.projection_metric(model(data_query), hyperplanes, mu=mu)
            loss = F.cross_entropy(logits, label) + args['lamb']*discriminative_loss
            acc = count_acc(logits, label)

            tl.add(loss.item())
            ta.add(acc)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Update per-epoch logs for each train_loader
        trlog[category]['train_loss'].append(tl.item())
        trlog[category]['train_acc'].append(ta.item())

        print(f'Epoch {epoch}, {category} Train Loss={tl.item():.4f}, Train Accuracy={ta.item():.4f}')

    # Validation phase
    model.eval()

    vl_total = Averager()  # For tracking overall validation loss
    va_total = Averager()  # For tracking overall validation accuracy

    # Loop over validation loaders and categories
    for category, val_loader in zip(val_categories, val_loaders):
        for i, batch in tqdm(enumerate(val_loader, 1), desc=f"Validating {category}"):
            data, label = [_.cuda() for _ in batch]  # Assuming `label` is provided with each batch

            p = args['shot'] * args['test-way']
            data_shot, data_query = data[:p], data[p:]
        
            if args['shot'] == 1:
                data_shot = torch.cat((data_shot, flip(data_shot, 3)), dim=0)
        
            proto = model(data_shot)
            proto = proto.reshape(shot_num, args['test-way'], -1)
            proto = torch.transpose(proto, 0, 1)
            hyperplanes, mu = projection_pro.create_subspace(proto, args['test-way'], shot_num)
        
            label = torch.arange(args['test-way']).repeat(args['query'])
            label = label.type(torch.cuda.LongTensor)
        
            logits, _ = projection_pro.projection_metric(model(data_query), hyperplanes, mu=mu)
            loss = F.cross_entropy(logits, label)
            acc = count_acc(logits, label)
        
            vl_total.add(loss.item())  # Add to overall validation loss tracker
            va_total.add(acc)         # Add to overall validation accuracy tracker

        vl_final = vl_total.item()
        va_final = va_total.item()

        # Update per-epoch validation logs for each category
        trlog[category]['val_loss'].append(vl_final)
        trlog[category]['val_acc'].append(va_final)

        print(f'Epoch {epoch}, {category} Val Loss={vl_final:.4f}, Val Accuracy={va_final:.4f}, Max Accuracy={trlog["max_acc"]:.4f}')

        if va_final > trlog['max_acc']:
            trlog['max_acc'] = va_final
            save_model(f'max-acc_{category}')
            trlog['max_epoch'] = epoch

    
    tel_total = Averager()  # For tracking overall validation loss
    tea_total = Averager()  # For tracking overall validation accuracy
    category = 'list'
    for i, batch in tqdm(enumerate(test_loader, 1), desc=f"Testing {category}"):
        data, label = [_.cuda() for _ in batch]  # Assuming `label` is provided with each batch

        p = args['shot'] * args['test-way']
        data_shot, data_query = data[:p], data[p:]
    
        if args['shot'] == 1:
            data_shot = torch.cat((data_shot, flip(data_shot, 3)), dim=0)
    
        proto = model(data_shot)
        proto = proto.reshape(shot_num, args['test-way'], -1)
        proto = torch.transpose(proto, 0, 1)
        hyperplanes, mu = projection_pro.create_subspace(proto, args['test-way'], shot_num)
    
        label = torch.arange(args['test-way']).repeat(args['query'])
        label = label.type(torch.cuda.LongTensor)
    
        logits, _ = projection_pro.projection_metric(model(data_query), hyperplanes, mu=mu)
        loss = F.cross_entropy(logits, label)
        acc = count_acc(logits, label)
    
        tel_total.add(loss.item())  # Add to overall validation loss tracker
        tea_total.add(acc)         # Add to overall validation accuracy tracker

    tel_final = tel_total.item()
    tea_final = tea_total.item()

    # Update per-epoch validation logs for each category
    trlog[category]['test_loss'].append(tel_final)
    trlog[category]['test_acc'].append(tea_final)

    print(f'Epoch {epoch}, {category} Test Loss={tel_final:.4f}, Test Accuracy={tea_final:.4f}')

    print(f'ETA: {timer.measure()}/{timer.measure(epoch / args["max-epoch"])}')

    # Save logs to trlog and model checkpoints
    torch.save(trlog, osp.join(args['save-path'], 'trlog'))

    save_model('epoch-last')

    if epoch % args['save-epoch'] == 0:
        save_model(f'epoch-{epoch}')

    

/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:216: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(
Training bare: 100it [02:23,  1.44s/it]


Epoch 1, bare Train Loss=1.2636, Train Accuracy=0.4828


Training menu: 100it [02:25,  1.45s/it]


Epoch 1, menu Train Loss=1.1155, Train Accuracy=0.5510


Training modal: 100it [02:13,  1.34s/it]


...

Epoch 20, form Val Loss=1.0942, Val Accuracy=0.7408, Max Accuracy=0.7902


Validating tutorial: 100it [02:16,  1.36s/it]


Epoch 20, tutorial Val Loss=1.0388, Val Accuracy=0.7720, Max Accuracy=0.7902


Testing list: 100it [02:21,  1.42s/it]


Epoch 20, list Test Loss=1.3201, Test Accuracy=0.7332
ETA: 10.5h/10.5h


In [15]:
# Make zip file to download train model result

import shutil

# Define the directory to be zipped and the output zip file path
directory_to_zip = '/kaggle/working/save/'
output_zip_file = '/kaggle/working/save.zip'

# Compress the directory into a zip file
shutil.make_archive(output_zip_file.replace('.zip', ''), 'zip', directory_to_zip)

print(f"Directory zipped successfully to {output_zip_file}")


Directory zipped successfully to /kaggle/working/save.zip
